<h1>Projeto De Previsao de Compras</h1>
<h2>Versao.: 1.0</h2>
<h2>Data.:   07/12/2018</h2>
<p>Criar um dataframe com os resultados e a media movel que foi adquirido</p>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from math import sqrt
from sklearn.metrics import mean_squared_error 

dateparse = lambda dates: pd.datetime.strptime(dates, '%d.%m.%Y %H:%M:%S')

#windows
#dataFrame = pd.read_csv(r"C:\Users\Paulo Cesar\Estudos\Data Science\Ajuste Media de Compra\VENDA-GAM-343.CSV", 
                        #sep=";", parse_dates = ['dt_venda'], index_col = 'dt_venda', date_parser = dateparse)

#Linux
dataFrame = pd.read_csv(r"/home/pc/Estudos/Data Science/Ajuste Media de Compra/VENDA-GAM-3.CSV", 
                        sep=";", parse_dates = ['dt_venda'], index_col = 'dt_venda', date_parser = dateparse)
    
#Windows
#dataFrame = pd.read_csv(r"C:\Users\Paulo Cesar\Estudos\Data Science\Ajuste Media de Compra\\VENDA-GAM-343.CSV",sep=";")

#Linux
#dataFrame = pd.read_csv(r"/home/pc/Estudos/Data Science/Ajuste Media de Compra/VENDA-GAM-343.CSV",sep=";")

#dataFrame['dt_venda'] = pd.to_datetime(dataFrame['dt_venda'])

#dataFrame.index = pd.to_datetime(dataFrame.index)


In [ ]:
#dataFrame.drop(index=dataFrame.loc[dataFrame['dt_venda'] == '2018-01-11'].index)
#df = df.drop(df[df.score < 50].index)
dataAtual = buscarDataAtual()
dataFrame = dataFrame[dataFrame.index != dataAtual]

In [ ]:
def buscarDataAtual():
    data_atual = datetime.today()
    data_em_texto = '{}-{}-{}'.format(data_atual.year, data_atual.month, '01')
    return data_em_texto

In [ ]:
def calcularErro(dataAtual):
    for i, df in enumerate(arrayDeDataFrame):
        if (len(df) > 6):
            aVendida = df.iloc[-6:, 3].values
            j = 1
            while j <= 12:
                aMM = df.iloc[-8:-2, j+2].values
                erroMM = sqrt(mean_squared_error(aVendida, aMM))
                df.loc[dataAtual, 'MM'+str(j)] = erroMM;
                j+=1

In [ ]:
arrayMercadoria = dataFrame['cd_mercadoria'].unique()
arrayEmpresa = [1, 4, 5]
#arrayMercadoria = [117661]
#arrayEmpresa = [1]

arrayDeDataFrame = []



for emp in arrayEmpresa:
    i = 0
    for i, mer in enumerate(arrayMercadoria):
        df = dataFrame[(dataFrame['cd_mercadoria'] == mer) & 
                      (dataFrame['cd_empresa'] == emp)].sort_values(by=['dt_venda'],ascending=True)
        
        if (len(df) > 0):
            idx = pd.date_range(min(df.index), dataAtual, freq='MS')
            df.index = pd.DatetimeIndex(df.index)
            df = df.reindex(idx, fill_value=0)
            
            arrayDeDataFrame.append(df)                
            
            j = 1
            while j <= 12:
                arrayDeDataFrame[len(arrayDeDataFrame)-1]['MM'+str(j)] = arrayDeDataFrame[len(arrayDeDataFrame)-1]['qt_vendida'].rolling(j).mean()
                j += 1
            
        arrayDeDataFrame[len(arrayDeDataFrame)-1].fillna(0, inplace=True)
         
calcularErro(dataAtual)

In [ ]:
arrayDeDataFrame[100]

In [ ]:
min(dict(arrayDeDataFrame[100].iloc[-1, 4:]), key=dict(arrayDeDataFrame[100].iloc[-1, 4:]).get)

In [ ]:
arrayDeDataFrame[100][min(dict(arrayDeDataFrame[100].iloc[-1, 4:]), 
                        key=dict(arrayDeDataFrame[100].iloc[-1, 4:]).get)][len(arrayDeDataFrame[100])-2]
